## Streamlit JOIN

In [1]:
#초기 설정및 시스템 라이브러리
import platform
import warnings

# 데이터 시각화 라이브러리
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
print(platform.system())
warnings.filterwarnings('ignore')

# 행,열,결과값 생략 없이 보기,세팅
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

# setting Korean font
import platform
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

# statistic
from scipy import stats
from scipy.stats import shapiro, levene, ttest_ind, chi2_contingency, f_oneway
from scipy.stats import mannwhitneyu, fisher_exact, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

from collections import Counter
from sklearn.datasets import load_diabetes
import scipy.stats as st
from pathlib import Path

import re

# setting seed
np.random.seed(42)
    
print("="*60)
print("라이브러리 로드 완료!")
print("한글 폰트 설정 완료!")
print("="*60)

Windows
라이브러리 로드 완료!
한글 폰트 설정 완료!


In [3]:
# Load Dataset
sf_master   = pd.read_csv("\sparta\Final_project\data\stat\sf_master.csv")              # sf_master
s_master    = pd.read_csv("\sparta\Final_project\data\stat\success_master.csv")         # success_master
obj         = pd.read_csv("\sparta\Final_project\data\clean\clean_objects_final.csv")    # clean_objects_final
inv         = pd.read_csv("\sparta\Final_project\data\clean\clean_investments_final.csv")    # clean_investments_final
off         = pd.read_csv("\sparta\Final_project\data\clean\clean_offices_final.csv")        # clean_offices_final

print("="*60)
print("데이터셋 로드 완료!")
print("="*60)

데이터셋 로드 완료!


<span style="font-size: 13px;">
grain: 1행 = 1스타트업 <br>
기준: "objects_cfpr_id"<br>
</span>

In [4]:
# -----------------------------
# 0) 공통 유틸
# -----------------------------
def _to_str(s: pd.Series) -> pd.Series:
    return s.astype("string").str.strip()

def _mode_first(s: pd.Series):
    s = s.dropna()
    if s.empty:
        return pd.NA
    m = s.mode()
    if m.empty:
        return pd.NA
    return sorted(m.astype(str).tolist())[0]

# -----------------------------
# 1) ID 컬럼 문자열 정리
# -----------------------------
obj["objects_cfpr_id"] = _to_str(obj["objects_cfpr_id"])
inv["investor_cfp_id"] = _to_str(inv["investor_cfp_id"])
inv["invested_c_id"]   = _to_str(inv["invested_c_id"])
off["offices_c_id"]    = _to_str(off["offices_c_id"])
s_master["objects_cfpr_id"] = _to_str(s_master["objects_cfpr_id"])

# sf_master에도 objects_cfpr_id  같이 정리
if "objects_cfpr_id" in sf_master.columns:
    sf_master["objects_cfpr_id"] = _to_str(sf_master["objects_cfpr_id"])

# -----------------------------
# 2) Investor-only 회사 목록 만들기
#    - 투자자(investor)로 등장한 c:
#    - 투자받은회사(invested)로 등장한 c:
#    - investor에만 있는 c:는 제거
# -----------------------------
c_investors = set(
    inv.loc[inv["investor_cfp_id"].str.startswith("c:", na=False), "investor_cfp_id"]
      .dropna().unique().tolist()
)

c_invested = set(
    inv.loc[inv["invested_c_id"].str.startswith("c:", na=False), "invested_c_id"]
      .dropna().unique().tolist()
)

c_only_investor = c_investors - c_invested

# -----------------------------
# 3) "스타트업(회사)" 기준 ID 만들기
#    - success_master에 등장하는 회사(c:)를 모집단으로 잡고
#    - investor-only를 제거하면, 통상 194,151이 됩니다.
# -----------------------------
base_company_ids = set(
    s_master.loc[s_master["objects_cfpr_id"].str.startswith("c:", na=False), "objects_cfpr_id"]
      .dropna().unique().tolist()
)

startup_ids = base_company_ids - c_only_investor

print("="*60)
print("[CHECK] success_master 회사(c:) unique:", len(base_company_ids))
print("[CHECK] investor-only 제거 대상:", len(c_only_investor))
print("[CHECK] startup_ids(=회사 - investor-only):", len(startup_ids))
print("="*60)

# -----------------------------
# 4) obj에서 스타트업만 남기기
# -----------------------------
obj_c = obj[obj["objects_cfpr_id"].str.startswith("c:", na=False)].copy()
obj_startups = obj_c[obj_c["objects_cfpr_id"].isin(startup_ids)].copy()

print("[CHECK] obj_startups rows:", len(obj_startups),
      "| nunique:", obj_startups["objects_cfpr_id"].nunique())

# -----------------------------
# 5) offices -> 회사별 대표 오피스(1행) 요약
#    - city/country/state는 최빈값
#    - office_id는 결측 제외 최소값(대표값)로 잡음
# -----------------------------
off_c = off[off["offices_c_id"].str.startswith("c:", na=False)].copy()
off_c = off_c[off_c["offices_c_id"].isin(startup_ids)].copy()

# 타입 정리
off_c["office_id"] = pd.to_numeric(off_c.get("office_id"), errors="coerce")
off_c["city"] = _to_str(off_c.get("city"))
off_c["country_code"] = _to_str(off_c.get("country_code"))
off_c["offices_state_filled"] = _to_str(off_c.get("offices_state_filled"))

office_city_final = (
    off_c.groupby("offices_c_id", as_index=False)
        .agg(
            n_offices=("office_id", "nunique"),
            office_city=("city", _mode_first),
            office_country=("country_code", _mode_first),
            office_state=("offices_state_filled", _mode_first),
            office_id=("office_id", lambda x: pd.to_numeric(x, errors="coerce").dropna().min()),
        )
)

# office_id Int64
office_city_final["office_id"] = pd.to_numeric(office_city_final["office_id"], errors="coerce").astype("Int64")

print("[CHECK] office_city_final rows:", len(office_city_final),
      "| nunique:", office_city_final["offices_c_id"].nunique())

# -----------------------------
# 6) startup_profile = obj_startups + office_city_final
#    - validate="1:1"로 조인 폭발 방지
# -----------------------------
startup_profile = obj_startups.merge(
    office_city_final,
    left_on="objects_cfpr_id",
    right_on="offices_c_id",
    how="left",
    validate="1:1"
)

# 조인 후 불필요 키 컬럼 정리(선택)
# startup_profile = startup_profile.drop(columns=["offices_c_id"], errors="ignore")

print("[CHECK] startup_profile rows:", len(startup_profile),
      "| nunique:", startup_profile["objects_cfpr_id"].nunique())

# -----------------------------
# 7) (중요) Streamlit용: 1행=1스타트업 s_master_distinct_startups 만들기
#    - 기존 success_master(라운드 포함)에서 startup_ids만 남기고
#    - 요구 집계 기준으로 groupby
# -----------------------------
sm = s_master[s_master["objects_cfpr_id"].isin(startup_ids)].copy()

# 날짜/숫자 타입 (안전)
for c in ["founded_at", "funded_at", "acquired_at", "first_public_at"]:
    if c in sm.columns:
        sm[c] = pd.to_datetime(sm[c], errors="coerce")

for c in ["funding_rounds","funding_total_usd","relationships","raised_amount_usd","funding_round_id","num_fr_type","success_flag","is_first_round","is_last_round"]:
    if c in sm.columns:
        sm[c] = pd.to_numeric(sm[c], errors="coerce")

sm["founded_year"] = sm["founded_at"].dt.year

# first/last raised
first_raised = (sm.loc[sm["is_first_round"] == 1]
                  .groupby("objects_cfpr_id")["raised_amount_usd"].max())
last_raised  = (sm.loc[sm["is_last_round"] == 1]
                  .groupby("objects_cfpr_id")["raised_amount_usd"].max())

# last stage(투자단계 last): is_last_round 우선
last_stage = (sm.loc[sm["is_last_round"] == 1]
                .sort_values(["objects_cfpr_id","num_fr_type","funding_round_id"], ascending=[True, False, False])
                .drop_duplicates("objects_cfpr_id", keep="first")
                .set_index("objects_cfpr_id")["cat_fr_type"])

# 라운드수: funding_rounds(max) vs funding_round_id nunique 중 큰 값
rounds_max = sm.groupby("objects_cfpr_id")["funding_rounds"].max().fillna(0)
rounds_n   = sm.groupby("objects_cfpr_id")["funding_round_id"].nunique().fillna(0)
total_rounds = np.maximum(rounds_max, rounds_n)

s_master_distinct_startups = (
    sm.groupby("objects_cfpr_id", as_index=False)
      .agg(
          industry=("obj_category_filled", "first"),
          country=("country_code", "first"),
          founded_year=("founded_year", "min"),
          total_funding_usd=("funding_total_usd", "max"),
          relationships=("relationships", "max"),
          success_flag=("success_flag", "max"),
          ipo_achieved=("ipo_id", lambda x: int(pd.Series(x).notna().any())),
          mna_achieved=("acquisition_id", lambda x: int(pd.Series(x).notna().any())),
      )
)

# 붙이기
s_master_distinct_startups = s_master_distinct_startups.set_index("objects_cfpr_id")
s_master_distinct_startups["total_rounds"] = total_rounds
s_master_distinct_startups["first_round_raised_usd"] = first_raised
s_master_distinct_startups["last_round_raised_usd"]  = last_raised
s_master_distinct_startups["invest_stage_last"]      = last_stage
s_master_distinct_startups = s_master_distinct_startups.reset_index()

print("[CHECK] s_master_distinct_startups rows:", len(s_master_distinct_startups),
      "| nunique:", s_master_distinct_startups["objects_cfpr_id"].nunique())

# -----------------------------
# 8) 저장(원하는 경로로 수정)
# -----------------------------
startup_profile.to_csv("\sparta\Final_project\data\stat\startup_profile.csv", index=False)
office_city_final.to_csv("\sparta\Final_project\data\stat\office_city_final.csv", index=False)
s_master_distinct_startups.to_csv("\sparta\Final_project\data\stat\s_master_distinct_startups.csv", index=False)

print("="*60)
print("전처리 저장 완료!")
print("- startup_profile.csv")
print("- office_city_final.csv")
print("- s_master_distinct_startups.csv")
print("="*60)


[CHECK] success_master 회사(c:) unique: 196549
[CHECK] investor-only 제거 대상: 2440
[CHECK] startup_ids(=회사 - investor-only): 194151
[CHECK] obj_startups rows: 194151 | nunique: 194151
[CHECK] office_city_final rows: 86895 | nunique: 86895
[CHECK] startup_profile rows: 194151 | nunique: 194151
[CHECK] s_master_distinct_startups rows: 194151 | nunique: 194151
전처리 저장 완료!
- startup_profile.csv
- office_city_final.csv
- s_master_distinct_startups.csv
